<a href="https://colab.research.google.com/github/and-rgr/contradiction_and_entailment/blob/main/dce_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers

In [23]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
pd.set_option('display.expand_frame_repr', False)

In [25]:
# set up the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [26]:
# load data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

na_train = len(train) - len(train.dropna())
na_test = len(test) - len(test.dropna())

print("check for missing rows:", "\n\ttrain data:", na_train, ", test data:", na_test, "\n")

# reduce data - optional
train = train.iloc[:100]
test = test.iloc[:50]

print("train data shape: ", train.shape)
print("test data shape: ", test.shape)

check for missing rows: 
	train data: 0 , test data: 0 

train data shape:  (100, 6)
test data shape:  (50, 5)


In [27]:
# from transformers import TFAutoModel,AutoTokenizer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# learning_rate = 1e-5
# batch_size = 32
# warmup = 600
# max_seq_length = 128
# num_train_epochs = 3.0

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [28]:
# is .batch_encode_plus needed here?
# tf_batch = tokenizer()

max_len = 100

tf_batch = tokenizer.batch_encode_plus(
    train[['premise','hypothesis']].values.tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    return_attention_mask=True,
    # return_tensors="tf"
)

In [29]:
train_tf1=tf.convert_to_tensor(tf_batch['input_ids'],dtype=tf.int32)
train_tf2=tf.convert_to_tensor(tf_batch['attention_mask'],dtype=tf.int32)
train_input={'input_word_ids':train_tf1,'input_mask':train_tf2}

In [30]:
### TOKENIZATION DONE ###

In [31]:
# # what does this do?
# # why training = True?
# tf_outputs = tf_model(tf_batch, training = True)

In [32]:
# tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

In [33]:
# If you have labels, you can provide them to the model, it will return a tuple with the loss and the final activations.
# tf_outputs = tf_model(tf_batch, labels = tf.constant([1, 0]))

In [34]:
### DistilBERT ###

In [35]:
with strategy.scope():
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")

    tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
    # tf_model = TFAutoModel.from_pretrained(model_name)

    embedding = tf_model([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation = 'softmax')(embedding)

    model = tf.keras.Model(inputs = [input_word_ids,input_mask], outputs = output)

    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-5),
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    
    model.summary()

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 100)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955010   ['input_word_ids[0][0]',         
 assification_1 (TFDistilBertFo  rOutput(loss=None,               'input_mask[0][0]']             
 rSequenceClassification)       logits=(None, 2),                                                 
                                 hidden_states=None                                         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [36]:
tf_model([input_word_ids, input_mask])[0]

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'tf_distil_bert_for_sequence_classification_1')>

In [37]:
# early_stop = tf.keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True, verbose=1, monitor='val_accuracy')

In [ ]:
# model.fit(train_input, train.label.values, epochs = 3, verbose = 1, batch_size = None, validation_split = 0.2, callbacks=[early_stop])

model.fit(train_input, train.label.values, epochs = 3, verbose = 1, batch_size = None, validation_split = 0.2)

Epoch 1/3
3/3 [==============================] - 66s 18s/step - loss: 1.1552 - accuracy: 0.3000 - val_loss: 1.1055 - val_accuracy: 0.4000
Epoch 2/3
